# Mount Google Drive

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Check if poem exist

In [ ]:
poem_file = "/content/drive/MyDrive/CMPE-297/Final_Project/WS_poem.pkl"
poet = "William_Shakespeare"

In [ ]:
import os.path
poem_exist = os.path.exists(poem_file)

In [ ]:
poem_exist

True

# Load the poem if exist

In [ ]:
import pickle
if poem_exist:
    with open(poem_file,"rb") as input_file:
        peom_list = pickle.load(input_file)

# Load Poem URL crawler

In [ ]:
def obj_iter(obj):
    for i in range(len(obj)):
        print(obj[i].text, obj[i]['href'])


def get_poem_url_json(poet, poem_num=50):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < poem_num:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 10:
                break
        page += 1
    print({"poet":poet, "URL":URL_list})
    return {"poet":poet, "URL":URL_list}

# Install NLP lib

In [ ]:
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 

# Poem URL to clean Poem

In [ ]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs(requestread,'html.parser')
        poem_content = (clean_txt(soup.find_all('div', class_="poem_body")[0].text))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_content])
    return poems_results

# Crawl and save poem

In [ ]:
if poem_exist == False:
    WS_pd=pd.DataFrame(scrapePoemFromURL(get_poem_url_json(poet, poem_num=30)['URL']), columns=["Title", "Peom"])

In [ ]:
if poem_exist == False:
    peom_list = WS_pd["Peom"].values.tolist()
    print(peom_list)

In [ ]:
if poem_exist == False:
    with open(poem_file,"wb") as output_file:
        pickle.dump(peom_list,output_file)

# Download and Install SJSU ML PoemGenerator API

In [ ]:
!pip install transformers==4.14.1
!pip install bitsandbytes-cuda111==0.26.0.post2
!pip install datasets==1.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!rm -rf 'sjsu_ml_lib.py'
!wget "https://drive.google.com/uc?export=download&id=1Xk5yPDnAd8USIF2IfY5V5K1l8njUxSQC&confirm=yes" -O "sjsu_ml_lib.py"

--2022-12-17 00:45:45--  https://drive.google.com/uc?export=download&id=1Xk5yPDnAd8USIF2IfY5V5K1l8njUxSQC&confirm=yes
Resolving drive.google.com (drive.google.com)... 173.194.192.101, 173.194.192.139, 173.194.192.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.192.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4hbbp1hj9iae51pmqsi66iscm5pcmtpr/1671237900000/17244733245378365086/*/1Xk5yPDnAd8USIF2IfY5V5K1l8njUxSQC?e=download&uuid=e8facde2-5886-401a-b978-1abea41a9a67 [following]
--2022-12-17 00:45:45--  https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4hbbp1hj9iae51pmqsi66iscm5pcmtpr/1671237900000/17244733245378365086/*/1Xk5yPDnAd8USIF2IfY5V5K1l8njUxSQC?e=download&uuid=e8facde2-5886-401a-b978-1abea41a9a67
Resolving doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.googleusercontent.com)... 172.2

In [ ]:
from sjsu_ml_lib import PoemGenerator

==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


# Fine-Tuning the model

In [ ]:
PoemGenerator(peom_list, max_length=-1, model_path="/content/model.pkl", model_save=True)

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.75G [00:00<?, ?B/s]

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

''

# Inference model

In [ ]:
new_poem = PoemGenerator(peom_list, start_words="All the world's a stage, And all the men and women merely players;", max_length=128, model_path="/content/model.pkl")
print(new_poem)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All the world's a stage, And all the men and women merely players; they have their exits and their entrances, And one man in his time plays many parts, His acts being seven ages. The first age is when He plays the boy, The second, the man, The third, the angry man; The fourth, the lover, And the fifth, the old man; And the sixth, the man of quality; And the seventh, the great man is played In every scene by the same man, And his age is the moment's thought It was Shakespeare. (Part I of II.iii.45-50)

